In [ ]:
%%time
import numpy as np
import tensorflow as tf
import cifar10                          
from PIL import Image

def main(argv=None):
    
  with tf.Graph().as_default() as g:  
    
    # Please change this file name
    # (このファイル名は変更してください。)
    img = Image.open('test.jpg').convert("RGB")
    
    # Resize
    img = img.resize((32,32), Image.LANCZOS)
    
    # raw
    raw = img.getdata()
    
    # One dimensional array 
    arr = np.array(raw)
    raw = arr.flatten()
    
    # R <-> B
    for i in range(32*32):
      w = raw[(i*3)]  
      raw[(i*3)] = raw[(i*3)+2]
      raw[(i*3)+2] = w
    image = tf.reshape(raw, [32,32,3])    
        
    # Crop the central [height, width] of the image.    
    resized_image = tf.image.resize_image_with_crop_or_pad(image, 24, 24)
    
    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(resized_image)  
    
    # Shaping
    images = tf.reshape(float_image,[1,24,24,3])

    # Inference .
    logits = cifar10.inference(images)
    
    # Softmax
    out = tf.nn.softmax(logits)

    # Restore
    variable_averages = tf.train.ExponentialMovingAverage(
        cifar10.MOVING_AVERAGE_DECAY)
    variables_to_restore = variable_averages.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)

    with tf.Session() as sess:
      ckpt = tf.train.get_checkpoint_state('train')
      if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        
        # Run
        result = sess.run([out])
        result = np.array(result)
        softmax = result.flatten()
        
        # Output
        print("Output:", np.round(softmax, 2))         
      else:
        print('No checkpoint file found')  
    
if __name__ == '__main__':
  tf.app.run()